In [1]:
import numpy as np
import os
import signal
import json

from parlai.core.agents import create_agent, create_agent_from_shared, get_agent_module
from parlai.core.worlds import create_task
from parlai.core.params import ParlaiParser
from parlai.core.utils import Timer, round_sigfigs, warn_once
from parlai.core.logs import TensorboardLogger
from parlai.scripts.build_dict import build_dict, setup_args as setup_dict_args
from parlai.core.distributed_utils import (
    sync_object, is_primary_worker, all_gather_list, is_distributed, num_workers
)
from parlai.scripts.build_pytorch_data import get_pyt_dict_file
from parlai.scripts.train_model import *

In [2]:
task = "personachat"
model = "seq2seq"
batch_size = 3
lr = 0.5
hidden_size = 128
args = f"""-m parlai.scripts.train_model -m {model} 
           -t {task} 
           -mf '/tmp/model' 
           -bs {batch_size} 
           -lr {lr} 
           -hs {hidden_size}"""

In [3]:
# Create dictionary of arguments
opt = setup_args().parse_args(args.split())

[ Main ParlAI Arguments: ] 
[  batchsize: 3 ]
[  datapath: C:\Users\snowy\Documents\6.864\final-project\Persona-Dialogue-Generation\data ]
[  datatype: train ]
[  download_path: C:\Users\snowy\Documents\6.864\final-project\Persona-Dialogue-Generation\downloads ]
[  hide_labels: False ]
[  image_mode: raw ]
[  multitask_weights: [1] ]
[  numthreads: 1 ]
[  show_advanced_args: False ]
[  task: personachat ]
[ ParlAI Model Arguments: ] 
[  dict_class: parlai.core.dict:DictionaryAgent ]
[  init_model: None ]
[  model: seq2seq ]
[  model_file: '/tmp/model' ]
[ Training Loop Arguments: ] 
[  dict_build_first: True ]
[  display_examples: False ]
[  eval_batchsize: None ]
[  evaltask: None ]
[  load_from_checkpoint: False ]
[  max_train_time: -1 ]
[  num_epochs: -1 ]
[  save_after_valid: False ]
[  save_every_n_secs: -1 ]
[  validation_cutoff: 1.0 ]
[  validation_every_n_epochs: -1 ]
[  validation_every_n_secs: -1 ]
[  validation_max_exs: -1 ]
[  validation_metric: accuracy ]
[  validation_met

In [4]:
# Agent is essentially an object for our 'Seq2Seq' model
agent = create_agent(opt)
# World is a BatchWorld object that stores the data and task at hand
# It contains a list (worlds) of DialogPartnerWorld objects
world = create_task(opt, agent)

[ no model with opt yet at: '/tmp/model'(.opt) ]
Dictionary: loading dictionary from '/tmp/model'.dict
[ num words =  18745 ]
[ Loading existing model params from '/tmp/model' ]
[creating task(s): personachat]
[loading fbdialog data:C:\Users\snowy\Documents\6.864\final-project\Persona-Dialogue-Generation\data\Persona-Chat\personachat\train_self_original.txt]


C:\Users\snowy\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\Users\snowy\Documents\6.864\final-project\Persona-Dialogue-Generation\parlai\core\torch_agent.py:730: UserWarning: LR scheduler is different from saved. Starting fresh!
  warn_once("LR scheduler is different from saved. Starting fresh!")


In [5]:
len(world.worlds) # = batch_size

3

Below I show how we run matches with "worlds" and "agents". I have made print statements inside parlai.core.worlds and modules to show what's happening internally. 

It seems agent 0 is simply data extraction and agent 1 is the model (receives observation from agent 0). The data is ordered such that batch 1 is persona + first dialogue line and batch two just provides the next observed lines (but as you can see from the input lines being printed for agent 1 the previous dialogue are still in the sequence). I'm pretty sure everything is saved in agent.batch_observations.

You can look in data/train_both_original.txt for the dialogue. 

In [7]:
# This is how we loop through batches
with world:
    b = 0
    while True:
        print("Batch =", b)
        b += 1
        # do one example / batch of examples
        world.parley()
        
        if b > 6:
            break

Batch = 0
agent: 0
agent: 1
**************************************************
Observation: your persona: my name is kevin.
your persona: i enjoy working out and learning new things.
your persona: i play the guitar.
your persona: i am studying software development.
your persona: i am a student in college.
hi ! i am upset that summer is over that is my favorite season .
Label: yeah it is sad that the summer is over , whats your name ? i am kevin .
**************************************************
**************************************************
Observation: your persona: i am on a competitive dance team.
your persona: i love to dance.
your persona: ariel is my favorite princess.
your persona: i am in high school.
hi i am okay tonight how are you doing ?
Label: i am good . just got done with my class .
**************************************************
**************************************************
Observation: your persona: my girlfriend lives in salt lake city.
your persona: i h

Batch = 5
agent: 0
agent: 1
**************************************************
Observation: your persona: my name is kevin.
your persona: i enjoy working out and learning new things.
your persona: i play the guitar.
your persona: i am studying software development.
your persona: i am a student in college.
hi ! i am upset that summer is over that is my favorite season .
yeah it is sad that the summer is over , whats your name ? i am kevin .
david . nice to meet you . my birthdays during june .
that is great , i was just getting ready to workout and learn some new things
sounds fun . i need to feed my cat . he is very hungry . my parents wo not help with that .
that is sad , i love playing the guitar maybe i can play for your cat .
i think he would really love that !
i could play for your cat in between classes at college . and you ?
i am currently working with computers .
awesome . i am studying software development .
cool that is a very productive field . i think you will do great !
La

In [8]:
# This is where dialogue is stored (same as above but concatenated)
print(world.batch_observations[1][0]['text'])

your persona: my name is kevin.
your persona: i enjoy working out and learning new things.
your persona: i play the guitar.
your persona: i am studying software development.
your persona: i am a student in college.
hi ! i am upset that summer is over that is my favorite season .
yeah it is sad that the summer is over , whats your name ? i am kevin .
david . nice to meet you . my birthdays during june .
that is great , i was just getting ready to workout and learn some new things
sounds fun . i need to feed my cat . he is very hungry . my parents wo not help with that .
that is sad , i love playing the guitar maybe i can play for your cat .
i think he would really love that !
i could play for your cat in between classes at college . and you ?
i am currently working with computers .
awesome . i am studying software development .
cool that is a very productive field . i think you will do great !
thank you , it was great getting to know you david . anything else going on with you ?
nothing

# Training Models

In [12]:
# All the exciting stuff happens in parlai.scripts.train_model, essentially we can just modify
# parameters in opt (batch size, lr, etc) and/or rewrite the training loop.
trainer = TrainLoop(opt)

[ building dictionary first... ]
[ dictionary already built .]
[ no model with opt yet at: '/tmp/model'(.opt) ]
Dictionary: loading dictionary from '/tmp/model'.dict
[ num words =  18745 ]
[ Loading existing model params from '/tmp/model' ]
[creating task(s): personachat]
[loading fbdialog data:C:\Users\snowy\Documents\6.864\final-project\Persona-Dialogue-Generation\data\Persona-Chat\personachat\train_self_original.txt]
[ training... ]


In [14]:
# We should probably delete the internal print statements ;) 
trainer.train()

agent: 0
agent: 1
**************************************************
Observation: your persona: my mother was a weightlifter.
your persona: i love money.
your persona: i have brown eyes.
your persona: my feet smell.
what have you been doing today ?
Label: working all day , my feet hurt and smell , but money is my favorite thing i love it
**************************************************
**************************************************
Observation: your persona: i teach middle school art.
your persona: i still like unicorns even though i m an adult.
your persona: i want to write a book before i die.
your persona: i would love to visit the pyramids.
your persona: i am excellent at playing pool.
good afternoon there how are you
Label: hi there , i am good today , just catching up and reading , you ?
**************************************************
**************************************************
Observation: your persona: i ride a skateboard.
your persona: i have often been called 

agent: 0
agent: 1
**************************************************
Observation: your persona: my mother was a weightlifter.
your persona: i love money.
your persona: i have brown eyes.
your persona: my feet smell.
what have you been doing today ?
working all day , my feet hurt and smell , but money is my favorite thing i love it
what do you do for work ? i agree money is a pretty sweet thing
i am a model , i have this bright brown eyes that people just love , you ?
i look pretty good too , since i do weight lifting
oh really ? my mom was a weightlifter
that is pretty awesome . i am actually in the olympics
wow , my mom never made it that far , i guess you can lift a lot
i bench 345 pounds . i am very strong
oh wow , that is strong
yes it is . where are you from ?
Label: new york city , that is where my modeling agency is
**************************************************
**************************************************
Observation: your persona: i teach middle school art.
your pers

[ time:225.0s total_exs:2264 epochs:0.03 ] {'exs': 21, 'lr': 0.5, 'num_updates': 8, 'loss': 61.46, 'token_acc': 0.1143, 'nll_loss': 8.77, 'ppl': 6437.0}
agent: 0
agent: 1
**************************************************
Observation: your persona: i want to be an explorer.
your persona: i am an athlete.
your persona: i visit india often.
your persona: i do not like my job.
hey whats up how are you
Label: watching tv thinking about how much i hate my job lol
**************************************************
**************************************************
Observation: your persona: i cry when i watch gone with the wind.
your persona: i like to play with clay models.
your persona: i am not a good artist.
your persona: i want to climb mount everest.
hi . how are you doing today ?
Label: i am doing good and yourself ?
**************************************************
**************************************************
Observation: your persona: i like all types of music.
your persona: 

agent: 0
agent: 1
**************************************************
Observation: your persona: i want to be an explorer.
your persona: i am an athlete.
your persona: i visit india often.
your persona: i do not like my job.
hey whats up how are you
watching tv thinking about how much i hate my job lol
haha well that sounds horrible . i love my job ! !
what do you do ? i am an athlete
well that sounds awesome ! ! ! i am a horse trainer
need a horse , i really want to explore the world
horses are amazing . i never get to sleep in though lol
last time i was on a hose i was in india
that is awesome ! aside from trying horses , i wish i knew my place here
i am not sure what you mean ?
just here on earth . my purpose . where i belong ya know ?
Label: yeah that is a tough one , i am not sure myself !
**************************************************
**************************************************
Observation: your persona: i cry when i watch gone with the wind.
your persona: i like to pla

[ time:228.0s total_exs:2294 epochs:0.03 ] {'exs': 30, 'lr': 0.5, 'num_updates': 18, 'loss': 82.58, 'token_acc': 0.1333, 'nll_loss': 8.258, 'ppl': 3858.0}
agent: 0
agent: 1
**************************************************
Observation: your persona: i like to play video games.
your persona: i am a musician.
your persona: my father used to be a mechanic.
your persona: i have been working the same job for two years.
hello how are you today ?
how are you doing today
i am good , just outside picking rocks up
i am good today
that is very good . do you like beans ?
yea its pretty nice out today
i am just staring at my paintings today
no i do not like beans
man i smell like them , you should come smell me
Label: um no thank you that is weird
**************************************************
**************************************************
Observation: your persona: i like rock music.
your persona: my name is gerald.
your persona: my favorite color is green.
your persona: i like basketball

agent: 0
agent: 1
**************************************************
Observation: your persona: i love being in nature.
your persona: my mother is a teacher.
your persona: i can speak three languages.
your persona: i like to bike.
hi , i really love my dog
Label: do you ? i have one , i love to bike
**************************************************
**************************************************
Observation: your persona: i like rock music.
your persona: my name is gerald.
your persona: my favorite color is green.
your persona: i like basketball.
hi how are you today ?
hello . i am great . i have lots of hobbies . how about you ?
i like to garden and exercise including running . you ?
rock music and basketball are my passions .
that is cool . do you have any children ? i have 3 !
no kids yet , still too young . do you work ?
i do , but i have a fairly long commute . probably about 300 miles per week .
that is long . do the kids help out in the garden ?
i wished that they did . . do

agent: 0
agent: 1
**************************************************
Observation: your persona: i love being in nature.
your persona: my mother is a teacher.
your persona: i can speak three languages.
your persona: i like to bike.
hi , i really love my dog
do you ? i have one , i love to bike
i i like red and i am short
blue is my favorite color can you speak any other languages
i can speak alien . only alien
really ? i do not speak alien . but wish i did
it is alot of fun . you should try .
i can really only ride a bike lol
oh ok . boring . i have a brother and sister
sick sick that is not boring at all
qwerty . very qwerty . i think so
Label: sick sick my mom is a teacher tho
**************************************************
**************************************************
Observation: your persona: one of my favorite shows is face off on say.
your persona: i like things in organized.
your persona: i am a stay at home mom.
your persona: i enjoy cooking but not baking.
your persona

[creating task(s): personachat]
[loading fbdialog data:C:\Users\snowy\Documents\6.864\final-project\Persona-Dialogue-Generation\data\Persona-Chat\personachat\valid_self_original.txt]
[ running eval: valid ]
agent: 0
agent: 1


KeyError: 'labels'